In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import csv
from textblob import TextBlob
import pandas
import sklearn
import cPickle
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import StratifiedKFold, cross_val_score, train_test_split 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.learning_curve import learning_curve

In [ ]:
messages = pandas.read_csv('./data/SMSSpamCollection', sep='\t', quoting=csv.QUOTE_NONE,
                           names=["label", "message"])
#print messages

In [ ]:
messages.groupby('label').describe()

In [ ]:
messages['length'] = messages['message'].map(lambda text: len(text))
print messages.head()

In [ ]:
messages.hist(column='length', by='label', bins=50)
plt.axis([0, 50, 0, 180], [0, 50, 0, 180])

In [ ]:
def split_into_tokens(message):
    message = unicode(message, 'utf8')  # convert bytes into proper unicode
    return TextBlob(message).words

In [ ]:
messages.message.head().apply(split_into_tokens)

In [ ]:
def split_into_lemmas(message):
    message = unicode(message, 'utf8').lower()
    words = TextBlob(message).words
    # for each word, take its "base form" = lemma 
    return [word.lemma for word in words]

messages.message.head().apply(split_into_lemmas)

In [ ]:
bow_transformer = CountVectorizer(analyzer=split_into_lemmas).fit(messages['message'])
print (bow_transformer.vocabulary_)

In [ ]:
vocab = bow_transformer.vocabulary_
q = np.empty([8874,1],dtype=str)

for key, value in vocab.iteritems():
    q[value-1] = key.decode("ascii")
    #print key
#vocab.to_csv('data.csv')

In [ ]:
messages_bow = bow_transformer.transform(messages['message'])
print 'sparse matrix shape:', messages_bow.shape
print 'number of non-zeros:', messages_bow.nnz
print 'sparsity: %.2f%%' % (100.0 * messages_bow.nnz / (messages_bow.shape[0] * messages_bow.shape[1]))

In [ ]:
tfidf_transformer = TfidfTransformer().fit(messages_bow)
print tfidf_transformer

In [ ]:
messages_tfidf = tfidf_transformer.transform(messages_bow)
messages_tfidf = messages_tfidf.toarray()
#print type(messages_tfidf)


#print messages_tfidf
#print messages_tfidf.shape

In [ ]:
std_tfidf=messages_tfidf.std(axis=0)
print std_tfidf

In [ ]:
avg_tfidf=messages_tfidf.mean(axis=0)

In [ ]:
#avg_sorted_tfidf = avg_tfidf.sort()
avg_sorted_tfidf = np.sort(avg_tfidf)
print avg_sorted_tfidf.shape

In [ ]:
print type(avg_sorted_tfidf)
temp = avg_sorted_tfidf[avg_sorted_tfidf>0.0001]
print temp.shape

In [ ]:
#print std_tfidf.shape
print avg_tfidf.shape

In [ ]:
print messages_tfidf[:]
print messages_tfidf[100].min(axis=0)

#type(messages_tfidf)

In [ ]:
print messages.message[0]

In [ ]:
X = messages_tfidf
y = messages.label

In [ ]:
print X.shape
print y.shape
print type(X)
print type(y)

In [ ]:
from sklearn import preprocessing
le_sex = preprocessing.LabelEncoder()

#to convert into numbers
y = le_sex.fit_transform(y)
print y

In [ ]:
X = X.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
params = logreg.fit(X_train, y_train)
#print type(params)
print logreg.coef_.size

In [ ]:
y_pred = logreg.predict(X_test)

In [ ]:
print y_pred

In [ ]:
from sklearn import metrics
print metrics.accuracy_score(y_test, y_pred)

<h1>heading</h1>